In [8]:
import pandas as pd
import numpy as np

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
import librosa

def extract_feature(file_name):
    audio_signal, sample_rate = librosa.load(file_name, sr=16000)

    spectogram = np.abs(librosa.stft(audio_signal, n_fft=512))
    spec_feature = np.mean(spectogram.T, axis=0)

    power_spectogram = spectogram ** 2
    mel = librosa.feature.melspectrogram(S=power_spectogram, sr=sample_rate)
    mel = librosa.power_to_db(mel)
    mel_feature = np.mean(mel.T, axis=0)

    mfcc = librosa.feature.mfcc(S=mel)
    mfcc_feature = np.mean(mfcc.T, axis=0)

    return spec_feature, mel_feature, mfcc_feature








In [11]:
# import librosa

# def extract_feature(file_name):
#     audio_signal, sample_rate = librosa.load(file_name, sr=16000)
#     spectogram = librosa.stft(audio_signal, n_fft=512)
#     spectogram = np.abs(spectogram)
#     spectogram_feature = np.mean(spectogram.T, axis=0)

#     power_spectogram = spectogram ** 2
#     mel = librosa.feature.melspectrogram(S=power_spectogram, sr=sample_rate)
#     mel = librosa.power_to_db(mel)
#     mel_feature = np.mean(mel.T, axis=0)

#     mfccs = librosa.feature.mfcc(S=mel, n_mfcc=20)
#     mfcc_feature = np.mean(mfccs.T, axis=0)

#     return spectogram_feature, mel_feature, mfcc_feature

In [12]:
from tqdm.notebook import tqdm

train_data = {'spec': [], 'mel': [], 'mfcc': []}

for file_name, accent in tqdm(zip(train.id, train.accent)):
    spec, mel, mfcc = extract_feature('train/' + accent + '/' + file_name)
    train_data['spec'].append(spec)
    train_data['mel'].append(mel)
    train_data['mfcc'].append(mfcc)

0it [00:00, ?it/s]

In [13]:
from tqdm.notebook import tqdm

test_data = {'spec': [], 'mel': [], 'mfcc': []}

for file_name in tqdm(test.id):
    spec, mel, mfcc = extract_feature('test/' + file_name)
    test_data['spec'].append(spec)
    test_data['mel'].append(mel)
    test_data['mfcc'].append(mfcc)

  0%|          | 0/120 [00:00<?, ?it/s]

In [14]:
from sklearn.ensemble import RandomForestClassifier

submit = pd.read_csv('sample_submit.csv')

for feature in train_data.keys():
    x = train_data[feature]
    test_x = test_data[feature]

    model = RandomForestClassifier(random_state=1)
    model.fit(x, train.accent)
    res = model.predict(test_x)
    submit.accent = res
    submit.to_csv(feature+'.csv', index=False)